In [1]:
"""
Entry script to generate step-by-step GSM8K answers with Qwen models.
This script loads GSM8K from Hugging Face, builds CoT prompts, and runs generation.
Configuration is defined via global variables (no CLI args).
"""

from typing import List

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from utils import extract_answer_from_response, check_answer_correctness, verify_step,extract_answer_robust


# ==================== Global Configuration ====================

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"
DATA_SPLIT = "test"  # train or test
LIMIT = 20  # number of samples to run; set None or 0 for full split
TEMPERATURE = 0.2
DEVICE = "cuda"


# Load GSM8K split with optional limit
# Args:
#   split: Dataset split name (train/test)
#   limit: Maximum number of samples to return
# Returns:
#   List of dataset rows
def load_gsm8k(split: str = "test", limit: int = 20):
    ds = load_dataset("gsm8k", "main", split=split)
    if limit and limit > 0:
        ds = ds.select(range(min(limit, len(ds))))
    return ds


# Build CoT prompts with simple instruction
def build_inputs(questions: List[str]) -> List[str]:
    return [f"\n\nQuestion: {q.strip()}Let's solve this step by step." for q in questions]


# Generate chain-of-thought completions
def generate_cot(model, tokenizer, prompts: List[str], max_tokens: int, temperature: float):
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=temperature > 0,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # Strip prompts from completions to keep only generated text
    completions = []
    for prompt, full_text in zip(prompts, decoded):
        completions.append(full_text[len(prompt):].strip())
    return completions

In [2]:

import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/root/autodl-tmp/huggingface/models'
os.environ['HF_DATASETS_CACHE'] = '/root/autodl-tmp/huggingface/datasets'


In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, padding_side='left')
# Ensure pad_token is set (often needed for batch generation)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)

dataset = load_gsm8k(split=DATA_SPLIT, limit=LIMIT)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
data = [dataset[i] for i in range(0,10) ]
MAX_TOKENS = 512

### self-explore and principle-generation

When failing, self-explore the first pit and then generate corresponding principle, retry with principle.

In [7]:
questions = [row["question"] for row in data]
ground_truth = [extract_answer_from_response(row["answer"]) for row in data]

# Principle Generation Prompt Template
generate_principle_prompt = """You are a helpful math assistant. 

### Question
{question}

### Student's Partial Reasoning
{partial_reasoning}

### The Error Step
{error_step}

### Right Answer
{answer}

### Instruction
The student failed to solve the problem correctly due to the error in the step above. 
Please provide a **generalized principle** to help the student understand the underlying concept and avoid similar mistakes.

### Steps to Generate the Principle
1. **Abstract the Problem**: Identify the general category of this problem (e.g., "calculating compound probabilities", "finding the remaining amount after multiple subtractions") without referring to specific numbers or items in this problem.
2. **Identify the Method**: Determine the correct general method or logical step required to solve this type of problem.
3. **Formulate the Hint**: Combine these into a single sentence.

### Requirements
- **Format**: Your output must be a single sentence following this pattern: "To solve [Abstract Problem Description], consider [General Method/Principle]."
- **Strict Constraint**: Do NOT include any specific numbers (e.g., 16, 3) or specific nouns (e.g., eggs, ducks) from the current problem. The principle must be universal. """

def generate_principle(model, tokenizer, question, partial_reasoning, error_step, answer):
    prompt = generate_principle_prompt.format(
        question=question,
        partial_reasoning=partial_reasoning,
        error_step=error_step,
        answer=answer
    )
    
    messages = [
        {"role": "system", "content": "You are a helpful math assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=256,
            temperature=0.7
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# Iterative Correction Loop
MAX_RETRIES = 3

correct_count = 0
for idx, (q, gt) in enumerate(zip(questions, ground_truth)):
    print(f"\n{'='*60}")
    print(f"{' Sample ' + str(idx) + ' ':=^60}")
    print(f"{'='*60}\n")

    print(f"{' Question ':-^60}")
    print(f"\n{q}\n")
    print(f"{' Ground Truth ':-^60}")
    print(f"\n{gt}\n")

    current_question_prompt = f"\n\nQuestion: {q.strip()}Let's solve this step by step."
    accumulated_principles = []
    
    for attempt in range(MAX_RETRIES + 1):
        print(f"\n--- Attempt {attempt + 1} ---")
        
        # If we have principles, prepend them to the prompt as hints
        if accumulated_principles:
            hints_str = "\n".join([f"Hint {i+1}: {p}" for i, p in enumerate(accumulated_principles)])
            # Re-construct prompt with hints
            # Note: We inject hints before the "Let's solve this step by step" instruction
            prompt_with_hints = f"Here are some hints to help you solve the problem:\n{hints_str}\n\nQuestion: {q.strip()}\nLet's solve this step by step."
            current_input_prompt = prompt_with_hints
        else:
            current_input_prompt = current_question_prompt

        # Generate CoT
        completions = generate_cot(model, tokenizer, [current_input_prompt], MAX_TOKENS, TEMPERATURE)
        comp = completions[0]
        
        # Check correctness
        pred_ans = extract_answer_robust(comp)
        is_correct = check_answer_correctness(pred_ans, gt)
        
        print(f"{' Completion ':-^60}")
        print(f"\n{comp}\n")
        print(f"{' Predicted Answer ':-^60}")
        print(f"\n{pred_ans}\n")
        print(f"✓ Correct" if is_correct else f"✗ Incorrect")
        
        if is_correct:
            correct_count += 1
            print(f"\n🎉 Solved correctly on attempt {attempt + 1}!")
            break
        
        # If incorrect and we have retries left, analyze and generate principle
        if attempt < MAX_RETRIES:
            print("\n--- 🔍 Self-Explore Error Analysis ---")
            steps = [s.strip() for s in comp.split('\n') if s.strip()]
            
            # Base prompt for verification (must match what was used for generation)
            verify_base_prompt = current_input_prompt
            
            first_wrong_step_idx = -1
            found_error = False
            
            for i, step in enumerate(steps):
                verify_base_prompt += "\n" + step
                print(f"Verifying Step {i+1}/{len(steps)}...", end=" ", flush=True)
                
                is_valid = verify_step(model, tokenizer, verify_base_prompt, gt, k=5) # Increased k for better verification
                
                if is_valid:
                    print("✅ Valid")
                else:
                    print(f"❌ Invalid (First Wrong Step Found)")
                    print(f"   -> Content: {step}")
                    first_wrong_step_idx = i
                    found_error = True
                    
                    # Generate Principle
                    print("\n--- 💡 Generating Principle for Correction ---")
                    partial_reasoning = "\n".join(steps[:i]) if i > 0 else "(No previous steps)"
                    
                    principle = generate_principle(
                        model, 
                        tokenizer, 
                        question=q, 
                        partial_reasoning=partial_reasoning, 
                        error_step=step, 
                        answer=gt
                    )
                    print(f"Generated Principle:\n{principle}")
                    accumulated_principles.append(principle)
                    break
            
            if not found_error:
                print("❓ Could not identify a specific wrong step. Retrying with full context...")

    if not is_correct:
        print(f"\n❌ Failed to solve after {MAX_RETRIES + 1} attempts.")

total = len(questions)
print(f"\nFinal Accuracy: {correct_count}/{total} = {correct_count/total:.2%}")


========================= Sample 0 =========================

------------------------- Question -------------------------

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

----------------------- Ground Truth -----------------------

18


--- Attempt 1 ---
------------------------ Completion ------------------------

First, we need to determine how many eggs Janet has left after using some for breakfast and baking muffins.

1. Calculate the total number of eggs laid per day:
   \[
   16 \text{ eggs}
   \]

2. Subtract the number of eggs Janet uses for breakfast:
   \[
   16 - 3 = 13 \text{ eggs}
   \]

3. Subtract the number of eggs used for baking muffins:
   \[
   13 - 4 = 9 \text{ eggs}
   \]

4. Determine the revenue from selling the remaining eggs at

## Baseline

In [6]:
# --- Baseline Experiment: Retry without Principles ---
print("\n" + "="*80)
print(f"{' BASELINE EXPERIMENT: RETRY WITHOUT PRINCIPLES ':=^80}")
print("="*80 + "\n")

baseline_correct_count = 0
MAX_RETRIES = 3

for idx, (q, gt) in enumerate(zip(questions, ground_truth)):
    print(f"\n{'='*60}")
    print(f"{' Sample ' + str(idx) + ' ':=^60}")
    print(f"{'='*60}\n")

    print(f"{' Question ':-^60}")
    print(f"\n{q}\n")
    print(f"{' Ground Truth ':-^60}")
    print(f"\n{gt}\n")

    # Standard Prompt (No Hints)
    current_input_prompt = f"\n\nQuestion: {q.strip()}Let's solve this step by step."
    
    for attempt in range(MAX_RETRIES + 1):
        print(f"\n--- Attempt {attempt + 1} ---")
        
        # Generate CoT (Temperature > 0 ensures diversity in retries)
        completions = generate_cot(model, tokenizer, [current_input_prompt], MAX_TOKENS, TEMPERATURE)
        comp = completions[0]
        
        # Check correctness
        pred_ans = extract_answer_robust(comp)
        is_correct = check_answer_correctness(pred_ans, gt)
        
        print(f"{' Completion ':-^60}")
        print(f"\n{comp}\n")
        print(f"{' Predicted Answer ':-^60}")
        print(f"\n{pred_ans}\n")
        print(f"✓ Correct" if is_correct else f"✗ Incorrect")
        
        if is_correct:
            baseline_correct_count += 1
            print(f"\n🎉 Solved correctly on attempt {attempt + 1}!")
            break
        else:
            if attempt < MAX_RETRIES:
                print("\n🔄 Incorrect. Retrying with a new sample (Temperature sampling)...")

    if not is_correct:
        print(f"\n❌ Failed to solve after {MAX_RETRIES + 1} attempts.")

total = len(questions)
print(f"\nBaseline Final Accuracy: {baseline_correct_count}/{total} = {baseline_correct_count/total:.2%}")


================ BASELINE EXPERIMENT: RETRY WITHOUT PRINCIPLES =================


========================= Sample 0 =========================

------------------------- Question -------------------------

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

----------------------- Ground Truth -----------------------

18


--- Attempt 1 ---
------------------------ Completion ------------------------

First, we need to determine how many eggs Janet has left after using some for breakfast and baking muffins.

1. Calculate the total number of eggs laid per day:
   \[
   16 \text{ eggs}
   \]

2. Subtract the number of eggs Janet uses for breakfast:
   \[
   16 - 3 = 13 \text{ eggs}
   \]

3. Subtract the number of eggs used for baking muffins:
   \[
   13 - 4 

In [8]:
del model